In [ ]:
def preprocess_text(text):
    """
    对文本进行预处理
    """
    text = text.lower()  # 转换为小写字母
    text = text.replace("'", "")  # 去除单引号
    text = text.replace("-", "")  # 去除破折号
    text = "".join([char for char in text if char.isalpha() or char.isspace()])  # 保留字母和空格
    return text

def tokenize_text(text):
    """
    将文本转换为tokens列表
    """
    tokens = text.split()
    return tokens

def plot_most_frequent_words(tokens, title):
    """
    绘制文本中出现频率最高的词汇柱状图
    """
    fdist = FreqDist(tokens)
    words_df = pd.DataFrame({'word': list(fdist.keys()), 'count': list(fdist.values())})
    # 选择出现频率最高的前20个词汇
    d = words_df.nlargest(columns="count", n=10)
    plt.figure(figsize=(16, 9))
    sns.barplot(data=d, x="word", y="count")
    plt.title(title)
    plt.show()


def plot_pearson_correlation_coefficient(X, data):
    """
    绘制各个文本之间的Pearson相关系数热力图
    """
    corr_matrix = np.zeros((len(data), len(data)))
    for i, j in itertools.combinations(range(len(data)), 2):
        corr, _ = pearsonr(X.toarray()[i], X.toarray()[j])
        corr_matrix[i, j] = corr
        corr_matrix[j, i] = corr
    tick_labels = data['title']
    sns.set(style="white")
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    f, ax = plt.subplots(figsize=(10, 8))
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5},
                xticklabels=tick_labels, yticklabels=tick_labels,
                annot=True, fmt='.2f', annot_kws={'size': 10})
    plt.title("Pearson Correlation Coefficient Matrix")
    plt.show()


def recommend_courses(sinput, df):
    # 引入CountVectorizer库，用于文本向量化
    vectorizer = CountVectorizer(tokenizer=tokenize_text, preprocessor=preprocess_text)

    # 将data中的文本转换成向量X，每行代表一个课程的向量表示
    X = vectorizer.fit_transform(df['text'])

    # 将用户输入的文本转换成向量表示
    student_input = vectorizer.transform(sinput.split())

    # 计算用户输入的文本和所有课程向量之间的余弦相似度
    cosine_sim = cosine_similarity(X, student_input)

    # 为每个课程在data中增加一列'similarity'，表示该课程与用户输入的文本的相似度得分
    df['similarity'] = cosine_sim.sum(1)

    # 根据'similarity'得分进行降序排列，选择前3个得分最高
    top_courses = df.sort_values('similarity', ascending=False).head(3)['Name']
    df = df.sort_values('similarity', ascending=False)
    #return top_courses.values
    return df